In [ ]:
# !wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

In [2]:
# import os
# import glob
# from zipfile import ZipFile 

# # download files and remove unnecessary ones
# with ZipFile("bank+marketing.zip", 'r') as zObject:
#     zObject.extractall()
# with ZipFile("bank.zip", 'r') as zObject:
#     zObject.extractall()

# file_paths = ['bank_marketing.ipynb', 'bank-full.csv']

# for clean_up in glob.glob("*.*"):
#     print(clean_up)
#     if not clean_up in file_paths:
#         os.remove(clean_up)

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [4]:
column_list = """age,
job,
marital,
education,
balance,
housing,
contact,
day,
month,
duration,
campaign,
pdays,
previous,
poutcome,
y""".split(',\n')
column_list

['age',
 'job',
 'marital',
 'education',
 'balance',
 'housing',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y']

In [5]:
df = pd.read_csv('bank-full.csv', sep=";")
df = df[column_list]
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no


In [6]:
df.isna().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   balance    45211 non-null  int64 
 5   housing    45211 non-null  object
 6   contact    45211 non-null  object
 7   day        45211 non-null  int64 
 8   month      45211 non-null  object
 9   duration   45211 non-null  int64 
 10  campaign   45211 non-null  int64 
 11  pdays      45211 non-null  int64 
 12  previous   45211 non-null  int64 
 13  poutcome   45211 non-null  object
 14  y          45211 non-null  object
dtypes: int64(7), object(8)
memory usage: 5.2+ MB


# Question 1
**Answer**: `secondary`

In [8]:
df['education'].mode()

0    secondary
Name: education, dtype: object

# Question 2
**Answer**: `pdays` and `previous`

In [9]:
# df.select_dtypes(int).columns
numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
df[numerical].corr().unstack().abs().sort_values().drop_duplicates()

duration  previous    0.001203
previous  age         0.001288
duration  pdays       0.001565
pdays     balance     0.003435
balance   day         0.004503
age       duration    0.004648
campaign  age         0.004760
day       age         0.009120
balance   campaign    0.014578
          previous    0.016674
duration  balance     0.021560
pdays     age         0.023758
day       duration    0.030206
campaign  previous    0.032855
day       previous    0.051710
duration  campaign    0.084570
campaign  pdays       0.088628
day       pdays       0.093044
age       balance     0.097783
day       campaign    0.162490
previous  pdays       0.454820
age       age         1.000000
dtype: float64

In [10]:
# target encoding
df['y'] = df['y'].replace({'yes': 1, 'no': 0})

/tmp/ipykernel_2204/3474783318.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['y'] = df['y'].replace({'yes': 1, 'no': 0})


In [22]:
# split the data
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.2, random_state=42)

X_train, y_train = df_train.drop('y', axis=1), df_train['y']
X_val, y_val = df_val.drop('y', axis=1), df_val['y']
X_test, y_test = df_test.drop('y', axis=1), df_test['y']

# Question 3
**Answer**: `poutcome`

In [12]:
categorical = [i for i in df.columns if i not in numerical]
categorical.remove('y')
categorical

['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

In [13]:
def calculate_mi(series, y=y_train):
    return mutual_info_score(series, y)

In [14]:
df_mi = X_train[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')
df_mi

,MI
poutcome,0.029389
month,0.024972
contact,0.013437
housing,0.010465
job,0.007172
education,0.002777
marital,0.002019


# Question 4
**Answer**: `0.9`

In [23]:
train_dict = X_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
X_train_matrix = dv.transform(train_dict)

In [24]:
val_dict = X_val[categorical + numerical].to_dict(orient='records')
X_val_matrix = dv.transform(val_dict)

In [25]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_matrix, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [26]:
y_pred = model.predict_proba(X_val_matrix)[:, 1]
y_hat = y_pred > 0.5
accuracy = (y_val == y_hat).mean()
round(accuracy, 2)

np.float64(0.9)

# Question 5
**Answer**: `balance`

In [51]:
all_features = categorical + numerical

def calculate_accuracy(features_list):
    train_dict = X_train[features_list].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dict)
    X_train_matrix = dv.transform(train_dict)

    val_dict = X_val[features_list].to_dict(orient='records')
    X_val_matrix = dv.transform(val_dict)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_matrix, y_train)

    y_pred = model.predict_proba(X_val_matrix)[:, 1]
    y_hat = y_pred > 0.5
    accuracy = (y_val == y_hat).mean()
    return accuracy

In [52]:
original_accuracy = calculate_accuracy(all_features)
original_accuracy

np.float64(0.9008847110865358)

In [54]:
result_dict = {}
for feature in all_features:
    current_features = [i for i in all_features if i!=feature]
    accuracy = calculate_accuracy(current_features)
    difference = np.abs(original_accuracy - accuracy)

    result_dict[feature] = float(difference)


In [60]:
{k: v for k, v in sorted(result_dict.items(), key=lambda item: item[1])}

{'contact': 0.0,
 'day': 0.00013823610727126123,
 'job': 0.00027647221454241144,
 'balance': 0.00027647221454241144,
 'previous': 0.0004147083218136727,
 'education': 0.0005529444290849339,
 'age': 0.0005529444290849339,
 'campaign': 0.0005529444290849339,
 'pdays': 0.0006911805363560841,
 'month': 0.0008294166436273454,
 'marital': 0.0011058888581697568,
 'housing': 0.0019353055017971021,
 'poutcome': 0.005944152612662457,
 'duration': 0.01147359690351113}

# Question 6
**Answer**: `0.1`

In [61]:
def calculate_accuracy(c):
    train_dict = X_train[categorical + numerical].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dict)
    X_train_matrix = dv.transform(train_dict)

    val_dict = X_val[categorical + numerical].to_dict(orient='records')
    X_val_matrix = dv.transform(val_dict)

    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train_matrix, y_train)

    y_pred = model.predict_proba(X_val_matrix)[:, 1]
    y_hat = y_pred > 0.5
    accuracy = (y_val == y_hat).mean()
    return accuracy

In [62]:
c_dict = {}
for c in [0.01, 0.1, 1, 10, 100]:
    accuracy = calculate_accuracy(c)
    c_dict[c] = accuracy

{k: v for k, v in sorted(c_dict.items(), key=lambda item: item[1])}

{0.01: np.float64(0.8989494055847387),
 1: np.float64(0.9008847110865358),
 10: np.float64(0.9012994194083495),
 100: np.float64(0.9017141277301631),
 0.1: np.float64(0.9021288360519768)}